<a href="https://colab.research.google.com/github/junior0428/GEE/blob/main/02_EDGEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [1]:

#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [3]:
image1=ee.Image(1)
#print(image1)
image1.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 1, 'min': 1, 'precision': 'int', 'type': 'PixelType'},
   'id': 'constant'}],
 'type': 'Image'}

In [4]:
#concatenar bandas 
image2=ee.Image(20)
image3=ee.Image.cat([image1, image2])
image3.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 1, 'min': 1, 'precision': 'int', 'type': 'PixelType'},
   'id': 'constant'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 20,
    'min': 20,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'constant_1'}],
 'type': 'Image'}

In [5]:
#otra forma de concatenar bandas 
imag=ee.Image([10,30])
imag.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 10,
    'min': 10,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'constant'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 30,
    'min': 30,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'constant_1'}],
 'type': 'Image'}

In [6]:
#Otra forma de concatenar bandas que es mas utilizado
image4=image1.addBands(imag)
image4.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 1, 'min': 1, 'precision': 'int', 'type': 'PixelType'},
   'id': 'constant'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 10,
    'min': 10,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'constant_1'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 30,
    'min': 30,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'constant_1_1'}],
 'type': 'Image'}

In [7]:
#rename el ID de cada imagen
image4.bandNames().getInfo()
image5=image4.rename(['img_01', 'img_02', 'img_3'])
image5.bandNames().getInfo()
image5.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 1, 'min': 1, 'precision': 'int', 'type': 'PixelType'},
   'id': 'img_01'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 10,
    'min': 10,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'img_02'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 30,
    'min': 30,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'img_3'}],
 'type': 'Image'}

In [8]:
#seleccionar las bandas de interes
image5.select(['img_3']).bandNames().getInfo()

['img_3']

##**Indices espectrales**

In [9]:
img=ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_008066_20200610')
#img.getInfo()
centr_img=img.geometry().centroid().coordinates().getInfo()# optenemos las coordenadas del centro
img.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']

In [10]:
mydicc_viz={'landsat8': img.getMapId({'bands':['B7', 'B5', 'B3'], 'max':0.5, 'min':0})}
mapdisplay(centr_img, mydicc_viz)

##**Normalized difference vegetation index (NDVI)**

In [11]:
#Metodo 1
nir=img.select(['B5'])
red=img.select(['B4'])
ndvi01=nir.subtract(red).divide(nir.add(red))
#Metodo 2
ndvi02=img.normalizedDifference(['B5','B4'])
ndvi_palette = ["#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001", "#006400", "#003B00", "#000000"]
mydicc = {
    'ndvi-method01': ndvi01.getMapId({'min':0,'max':0.8,'palette':ndvi_palette}),
    'ndvi-method02': ndvi02.getMapId({'min':-0.1,'max':0.8,'palette':ndvi_palette}),
}
mapdisplay(centr_img, mydicc)

##**Normalized difference water index (NDWI)**

In [12]:
ndwi=img.normalizedDifference(['B5', 'B6'])
ndwi_palette = ["#000180", "#0075FD", "#6CFB93", "#F99D05", "#A70700"]
ndwi_viz={'ndwi_huascaran':ndwi.getMapId({'min':0, 'max':1, 'palette':ndwi_palette})}
mapdisplay(center=centr_img, dicc=ndwi_viz)


##**Mascara**

In [13]:
ndwi=img.normalizedDifference(['B5', 'B6'])
ndwi_mask=ndwi.gt([0.4])
mapdisplay(centr_img, {'mask_ndwi':ndwi_mask.getMapId()})

In [14]:
ndwi_m=ndwi.updateMask(ndwi_mask)
ndwi_m_viz={'ndwi_huascaran':ndwi_m.getMapId({'min':0, 'max':1, 'palette':ndwi_palette})}
mapdisplay(center=centr_img, dicc=ndwi_m_viz)

##**Cortar una imagen**

In [15]:
ee_poly=ee.Geometry.Point([-77.623591, -9.086664]).buffer(5000)
ndwi_m_clip=ndwi_m.clip(ee_poly)
ndwi_m_clip_viz={'ndwi_huascaran':ndwi_m_clip.getMapId({'min':0, 'max':1, 'palette':ndwi_palette})}
mapdisplay(center=centr_img, dicc=ndwi_m_clip_viz)

##**Como exportar una imagen**

In [16]:
ndwi.getInfo()

{'bands': [{'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 139185, 0, -30, -844485],
   'data_type': {'max': 1,
    'min': -1,
    'precision': 'float',
    'type': 'PixelType'},
   'dimensions': [7581, 7721],
   'id': 'nd'}],
 'properties': {'system:footprint': {'coordinates': [[-78.13848414743335,
     -9.394701079166925],
    [-77.421067286116, -9.549598179747889],
    [-76.58899556018864, -9.727317136474108],
    [-76.58737076150426, -9.727324686516829],
    [-76.58360573068515, -9.711775779240638],
    [-76.50202772122984, -9.344319064595483],
    [-76.48699067180704, -9.276024184405221],
    [-76.32038787624258, -8.499081765586427],
    [-76.28465484548147, -8.331090464241743],
    [-76.2129451533772, -7.993119911929218],
    [-76.21293958276816, -7.991227052542581],
    [-76.5809674716001, -7.913198213233902],
    [-77.31171282704453, -7.7572656000401325],
    [-77.89023141756557, -7.632897839947056],
    [-77.89836822522199, -7.631217399778002],
    [-77.90396067461472, -7.65

In [17]:
ndwi_to_export=ndwi.multiply(1000).toInt()

In [23]:
#ee.batch.Export.image.toDrive()
from ee.batch import Export
task=Export.image.toDrive(image=ndwi_to_export, 
                          description='NDWI_Huascaran2',
                          folder='MASTERGIS_GEE', 
                          region=ee_poly, 
                          scale=250)
task

<Task EXPORT_IMAGE: NDWI_Huascaran2 (UNSUBMITTED)>

In [24]:
#Para comenzar la descargar hacia Drive
task.start()

###**Monitoreo de tareas**
OBS:**while** es un ciclo de preverificación

In [25]:
import time 
def ee_monitoring(ee_task):
  while ee_task.active():
    print('Sondeo de la tarea (id: {})'.format(ee_task.id))
    time.sleep(5)

In [26]:
ee_monitoring(task)

Sondeo de la tarea (id: VPIGM4CDFBQWSQM43KZA625V)
Sondeo de la tarea (id: VPIGM4CDFBQWSQM43KZA625V)
Sondeo de la tarea (id: VPIGM4CDFBQWSQM43KZA625V)
Sondeo de la tarea (id: VPIGM4CDFBQWSQM43KZA625V)
Sondeo de la tarea (id: VPIGM4CDFBQWSQM43KZA625V)
